<a href="https://colab.research.google.com/github/Derinhelm/graph_syntax_parsing/blob/main/Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00


In [2]:
!pip install torch_geometric

from IPython.display import clear_output

clear_output()

# Logging

In [49]:

import logging
logger = logging.getLogger('my_logger')

# Remove all handlers associated with the root logger object.
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='app.log', # write to this file
    filemode='a', # open in append mode
    format='%(name)s - %(levelname)s - %(message)s'
    )

logging.getLogger().setLevel(logging.INFO)
logging.getLogger().setLevel(logging.DEBUG)

logging.getLogger("urllib3.connectionpool").disabled = True
logging.getLogger("filelock").disabled = True

logging.warning('This will get logged to a file')

In [50]:
logging.warning('New warning')
logging.debug('New debug')
logging.info('New info')


In [51]:
from pathlib import Path
print(Path('/content/app.log').read_text())

root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info



#uuparser/utils.py

In [52]:
from collections import defaultdict, Counter
import re
import os,time
from operator import itemgetter
import random
import json
import pathlib
import subprocess
import sys

import tqdm



class ConllEntry:
    def __init__(self, id, form, lemma, pos, cpos, feats=None, parent_id=None, relation=None,
        deps=None, misc=None):

        self.id = id
        self.form = form
        self.cpos = cpos
        self.pos = pos
        self.parent_id = parent_id
        self.relation = relation

        self.lemma = lemma
        self.feats = feats
        self.deps = deps
        self.misc = misc

        self.pred_parent_id = None
        self.pred_relation = None

        self.pred_pos = None
        self.pred_cpos = None


    def __str__(self):
        '''values = [str(self.id), self.form, self.lemma, \
                  self.pred_cpos if self.pred_cpos else self.cpos,\
                  self.pred_pos if self.pred_pos else self.pos,\
                  self.feats, str(self.pred_parent_id) if self.pred_parent_id \
                  is not None else str(self.parent_id), self.pred_relation if\
                  self.pred_relation is not None else self.relation, \
                  self.deps, self.misc]
        return '\t'.join(['_' if v is None else v for v in values])'''
        return self.form + " " + str(self.id)

class ParseForest:
    def __init__(self, sentence):
        self.roots = list(sentence)

        for root in self.roots:
            root.children = []
            root.scores = None
            root.parent = None
            root.pred_parent_id = None
            root.pred_relation = None
            root.vecs = None

    def __len__(self):
        return len(self.roots)


    def Attach(self, parent_index, child_index):
        parent = self.roots[parent_index]
        child = self.roots[child_index]

        child.pred_parent_id = parent.id
        del self.roots[child_index]

    def __str__(self):
        return " ".join(map(str, self.roots))


def isProj(sentence):
    forest = ParseForest(sentence)
    unassigned = {entry.id: sum([1 for pentry in sentence if pentry.parent_id == entry.id]) for entry in sentence}

    for _ in xrange(len(sentence)):
        for i in xrange(len(forest.roots) - 1):
            if forest.roots[i].parent_id == forest.roots[i+1].id and unassigned[forest.roots[i].id] == 0:
                unassigned[forest.roots[i+1].id]-=1
                forest.Attach(i+1, i)
                break
            if forest.roots[i+1].parent_id == forest.roots[i].id and unassigned[forest.roots[i+1].id] == 0:
                unassigned[forest.roots[i].id]-=1
                forest.Attach(i, i+1)
                break

    return len(forest.roots) == 1


def get_irels(data):
    """
    Collect frequencies of words, cpos, pos and deprels + languages.
    """

    # could use sets directly rather than counters for most of these,
    # but having the counts might be useful in the future or possibly for debugging etc
    relCount = Counter()

    for sentence in data:
        for node in sentence:
            if isinstance(node, ConllEntry):
                relCount.update([node.relation])

    return list(relCount.keys())


def generate_root_token():
    return ConllEntry(0, '*root*', '*root*', 'ROOT-POS', 'ROOT-CPOS', '_', -1,
        'rroot', '_', '_')


def read_conll(filename, drop_nproj=False, train=True):
    fh = open(filename,'r',encoding='utf-8')
    logging.info(f"Reading {filename}")
    ts = time.time()
    dropped = 0
    sents_read = 0
    sentences = []
    tokens = [generate_root_token()]
    words = [] # all words from the dataset
    for line in fh:
        tok = line.strip().split('\t')
        if not tok or line.strip() == '': # empty line, add sentence to list or yield
            if len(tokens) > 1:
                sents_read += 1
                conll_tokens = [t for t in tokens if isinstance(t,ConllEntry)]
                if not drop_nproj or isProj(conll_tokens): # keep going if it's projective or we're not dropping non-projective sents
                    if train:
                        inorder_tokens = inorder(conll_tokens)
                        for i,t in enumerate(inorder_tokens):
                            t.projective_order = i
                        for tok in conll_tokens:
                            tok.rdeps = [i.id for i in conll_tokens if i.parent_id == tok.id]
                            if tok.id != 0:
                                tok.parent_entry = [i for i in conll_tokens if i.id == tok.parent_id][0]
                    sentences.append(tokens)
                else:
                    logging.debug('Non-projective sentence dropped')
                    dropped += 1
            tokens = [generate_root_token()]
        else:
            if line[0] == '#' or '-' in tok[0] or '.' in tok[0]: # a comment line, add to tokens as is
                tokens.append(line.strip())
            else: # an actual ConllEntry, add to tokens
                if tok[2] == "_":
                    tok[2] = tok[1].lower()
                words.append(tok[2])
                token = ConllEntry(int(tok[0]), tok[1], tok[2], tok[4], tok[3], tok[5], int(tok[6]) if tok[6] != '_' else -1, tok[7], tok[8], tok[9])

                tokens.append(token)

# deal with case where there are still tokens, that aren`t in sentences list
# e.g. when there is no newline at end of file
    if len(tokens) > 1:
        sentences.append(tokens)

    logging.debug(f'{sents_read} sentences read')

    te = time.time()
    logging.info(f'Time: {te-ts:.2g}s')
    return sentences, words


def write_conll(fn, conll_gen):
    logging.info(f"Writing to {fn}")
    sents = 0
    with open(fn, 'w', encoding='utf-8') as fh:
        for sentence in conll_gen:
            sents += 1
            for entry in sentence[1:]:
                fh.write(str(entry) + '\n')
            fh.write('\n')
        logging.debug(f"Wrote {sents} sentences")


numberRegex = re.compile("[0-9]+|[0-9]+\\.[0-9]+|[0-9]+[0-9,]+");
def normalize(word):
    return 'NUM' if numberRegex.match(word) else word.lower()


def inorder(sentence):
    queue = [sentence[0]]
    def inorder_helper(sentence,i):
        results = []
        left_children = [entry for entry in sentence[:i] if entry.parent_id == i]
        for child in left_children:
            results += inorder_helper(sentence,child.id)
        results.append(sentence[i])

        right_children = [entry for entry in sentence[i:] if entry.parent_id == i ]
        for child in right_children:
            results += inorder_helper(sentence,child.id)
        return results
    return inorder_helper(sentence,queue[0].id)


def set_seeds():
    python_seed = 1
    logging.debug("Using default Python seed")
    random.seed(python_seed)


def generate_seed():
    return random.randint(0,10**9) # this range seems to work for Dynet and Python's random function


# uuparser/multilayer_perceptron.py


In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import SAGEConv, to_hetero
import torch

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x




# uuparser/arc_hybrid.py

In [54]:
from operator import itemgetter
from itertools import chain
import time, random
import numpy as np
from copy import deepcopy
from collections import defaultdict
import json

import torch
import torch.nn as nn
import torch.optim as optim

import tqdm


from jax.numpy import int32
from torch_geometric.data import HeteroData

from transformers import AutoTokenizer, BertModel
import torch

In [55]:
def get_embed(tokenizer, model, word):

    inputs = tokenizer(word, return_tensors="pt")
    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state[0][0]
    return last_hidden_states.detach().cpu()


def get_embed_for_sentence(sentence):
    word_embeds = torch.empty((len(sentence), 768))
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")
    for i in range(len(sentence)):
        word_embeds[i] = get_embed(tokenizer, model, sentence[i].form)
    return word_embeds

def create_stack_edges(stack):
    if len(stack) == 0:
        return torch.stack((torch.tensor([], dtype=torch.int32), torch.tensor([], dtype=torch.int32)), dim=0)
    stack_edges = []
    if len(stack) == 1:
        stack_edges.append((stack[0].id - 1, stack[0].id - 1)) # temporary solution
    else:
        for i in range(len(stack) - 1): # Represents every two consecutive stack nodes as an edge
            stack_edges.append((stack[i].id - 1, stack[i + 1].id - 1))
    stack_edges = tuple(zip(*stack_edges))
    stack_edges = [torch.tensor(stack_edges[0]), torch.tensor(stack_edges[1])]
    return torch.stack(stack_edges, dim=0)

def create_buffer_edges(buffer):
    if len(buffer) == 0 or len(buffer) == 1: # Last element is a technical root element.
        return torch.stack((torch.tensor([], dtype=torch.int32), torch.tensor([], dtype=torch.int32)), dim=0)
    buffer_edges = []
    if len(buffer) == 2: # Last element is a technical root element.
        buffer_edges.append((buffer[0].id - 1, buffer[0].id - 1)) # temporary solution
    else:
        for i in range(len(buffer) - 2): # Last element is a technical root element.
        # Represents every two consecutive buffer nodes as an edge
            buffer_edges.append((buffer[i].id - 1, buffer[i + 1].id - 1))
    buffer_edges = tuple(zip(*buffer_edges))
    buffer_edges = [torch.tensor(buffer_edges[0]), torch.tensor(buffer_edges[1])]
    return torch.stack(buffer_edges, dim=0)

def create_graph_edges(sentence):
    graph_edges = []
    for node in sentence:
        if node.pred_parent_id is not None and node.pred_parent_id != 0 and node.pred_parent_id != -1:
            graph_edges.append((node.pred_parent_id - 1, node.id - 1))
    if len(graph_edges) == 0:
        return torch.stack((torch.tensor([], dtype=torch.int32), torch.tensor([], dtype=torch.int32)), dim=0)
    graph_edges = tuple(zip(*graph_edges))
    graph_edges = [torch.tensor(graph_edges[0]), torch.tensor(graph_edges[1])]
    return torch.stack(graph_edges, dim=0)

def config_to_graph(sentence, stack, buffer):
    word_embeds = get_embed_for_sentence(sentence)

    data = HeteroData()
    data['node']['x'] = word_embeds

    data[('node', 'graph', 'node')].edge_index = create_graph_edges(sentence)
    data[('node', 'stack', 'node')].edge_index = create_stack_edges(stack)
    data[('node', 'buffer', 'node')].edge_index = create_buffer_edges(buffer)
    return data

class Configuration:
    def __init__(self, sentence, irels):
        self.sentence = deepcopy(sentence) # ensures we are working with a clean copy of sentence and allows memory to be recycled each time round the loop
        self.sentence = [entry for entry in self.sentence if isinstance(entry, ConllEntry)]
        self.sentence = self.sentence[1:] + [self.sentence[0]]
        self.stack = ParseForest([])
        self.buffer = ParseForest(self.sentence)
        for root in self.sentence:
            root.relation = root.relation if root.relation in irels else 'runk'

    def get_stack(self):
        return self.stack

    def get_buffer(self):
        return self.buffer

    def get_sentence(self):
        return self.sentence

    def is_end(self):
        return len(self.buffer) == 1 and len(self.stack) == 0

    def __str__(self):
        return "stack:" + str(self.stack) + "\n" + "buffer:" + str(self.buffer) + "\n"

In [56]:

class ArcHybridLSTM:
    def __init__(self, options, out_irels_dims):

        global LEFT_ARC, RIGHT_ARC, SHIFT, SWAP
        LEFT_ARC, RIGHT_ARC, SHIFT, SWAP = 0,1,2,3

        self.oracle = options["oracle"]

        self.hidden_dims = options["hidden_dims"]
        self.out_irels_dims = out_irels_dims

        self.metadata = (['node'], [('node', 'graph', 'node'), ('node', 'stack', 'node'), ('node', 'buffer', 'node')])
        self.unlabeled_GNN = GNN(hidden_channels=self.hidden_dims, out_channels=4)
        self.unlabeled_GNN = to_hetero(self.unlabeled_GNN, self.metadata, aggr='sum')

        self.labeled_GNN = GNN(hidden_channels=self.hidden_dims, out_channels=2*self.out_irels_dims+2)
        self.labeled_GNN = to_hetero(self.labeled_GNN, self.metadata, aggr='sum')

        self.unlabeled_optimizer = optim.Adam(self.unlabeled_GNN.parameters(), lr=options["learning_rate"])
        self.labeled_optimizer = optim.Adam(self.labeled_GNN.parameters(), lr=options["learning_rate"])


    def __evaluate(self, config, train, irels):
        """
        ret = [left arc,
               right arc
               shift]

        RET[i] = (rel, transition, score1, score2) for shift, l_arc and r_arc
         shift = 2 (==> rel=None) ; l_arc = 0; r_acr = 1

        ret[i][j][2] ~= ret[i][j][3] except the latter is a dynet
        expression used in the loss, the first is used in rest of training
        """

        stack = config.get_stack()
        buf = config.get_buffer()
        sentence = config.get_sentence()
        graph = config_to_graph(sentence, stack.roots, buf.roots)
        output = self.unlabeled_GNN(graph.x_dict, graph.edge_index_dict)
        output = torch.sum(output['node'], dim=0)
        routput = self.labeled_GNN(graph.x_dict, graph.edge_index_dict)
        routput = torch.sum(routput['node'], dim=0)

        #scores, unlabeled scores
        scrs, uscrs = routput, output

        #transition conditions
        left_arc_conditions = len(stack) > 0
        right_arc_conditions = len(stack) > 1
        shift_conditions = buf.roots[0].id != 0
        swap_conditions = len(stack) > 0 and stack.roots[-1].id < buf.roots[0].id

        if not train:
            #(avoiding the multiple roots problem: disallow left-arc from root
            #if stack has more than one element
            left_arc_conditions = left_arc_conditions and not (buf.roots[0].id == 0 and len(stack) > 1)

        uscrs0, uscrs1, uscrs2, uscrs3 = uscrs[0], uscrs[1], uscrs[2], uscrs[3]

        if train:
            output0, output1, output2, output3 = output[0], output[1], output[2], output[3]


            ret = [ [ (rel, LEFT_ARC, scrs[2 + j * 2] + uscrs2, routput[2 + j * 2 ] + output2) for j, rel in enumerate(irels) ] if left_arc_conditions else [],
                   [ (rel, RIGHT_ARC, scrs[3 + j * 2] + uscrs3, routput[3 + j * 2 ] + output3) for j, rel in enumerate(irels) ] if right_arc_conditions else [],
                   [ (None, SHIFT, scrs[0] + uscrs0, routput[0] + output0) ] if shift_conditions else [] ,
                    [ (None, SWAP, scrs[1] + uscrs1, routput[1] + output1) ] if swap_conditions else [] ]
        else:
            s1,r1 = max(zip(scrs[2::2],irels))
            s2,r2 = max(zip(scrs[3::2],irels))
            s1 = s1 + uscrs2
            s2 = s2 + uscrs3
            ret = [ [ (r1, LEFT_ARC, s1) ] if left_arc_conditions else [],
                   [ (r2, RIGHT_ARC, s2) ] if right_arc_conditions else [],
                   [ (None, SHIFT, scrs[0] + uscrs0) ] if shift_conditions else [] ,
                    [ (None, SWAP, scrs[1] + uscrs1) ] if swap_conditions else [] ]
        return ret

    def Load(self, epoch):
        unlab_path = 'model_unlab' + '_' + str(epoch)
        lab_path = 'model_lab' + '_' + str(epoch)

        self.unlabeled_GNN = GNN(hidden_channels=self.hidden_dims, out_channels=4)
        self.labeled_GNN = GNN(hidden_channels=self.hidden_dims, out_channels=2*self.out_irels_dims+2)

        unlab_checkpoint = torch.load(unlab_path)
        self.unlabeled_GNN.load_state_dict(unlab_checkpoint['model_state_dict'])


        self.unlabeled_GNN = to_hetero(self.unlabeled_GNN, self.metadata, aggr='sum')
        self.labeled_GNN = to_hetero(self.labeled_GNN, self.metadata, aggr='sum')

        lab_checkpoint = torch.load(lab_path)
        self.labeled_GNN.load_state_dict(lab_checkpoint['model_state_dict'])


    def Save(self, epoch):
        unlab_path = 'model_unlab' + '_' + str(epoch)
        lab_path = 'model_lab' + '_' + str(epoch)
        logging.info(f'Saving unlabeled model to {unlab_path}')
        torch.save({'epoch': epoch, 'model_state_dict': self.unlabeled_GNN.state_dict()}, unlab_path)
        logging.info(f'Saving labeled model to {lab_path}')
        torch.save({'epoch': epoch, 'model_state_dict': self.labeled_GNN.state_dict()}, lab_path)

    def apply_transition(self, best, config):
        stack = config.get_stack()
        buf = config.get_buffer()
        if best[1] == SHIFT:
            stack.roots.append(buf.roots[0])
            del buf.roots[0]

        elif best[1] == SWAP:
            child = stack.roots.pop()
            buf.roots.insert(1,child)

        elif best[1] == LEFT_ARC:
            child = stack.roots.pop()
            parent = buf.roots[0]

        elif best[1] == RIGHT_ARC:
            child = stack.roots.pop()
            parent = stack.roots[-1]

        if best[1] == LEFT_ARC or best[1] == RIGHT_ARC:
            #attach
            child.pred_parent_id = parent.id
            child.pred_relation = best[0]

    def calculate_cost(self,scores,s0,s1,b,beta,stack_ids):
        if len(scores[LEFT_ARC]) == 0:
            left_cost = 1
        else:
            left_cost = len(s0[0].rdeps) + int(s0[0].parent_id != b[0].id and s0[0].id in s0[0].parent_entry.rdeps)


        if len(scores[RIGHT_ARC]) == 0:
            right_cost = 1
        else:
            right_cost = len(s0[0].rdeps) + int(s0[0].parent_id != s1[0].id and s0[0].id in s0[0].parent_entry.rdeps)


        if len(scores[SHIFT]) == 0:
            shift_cost = 1
            shift_case = 0
        elif len([item for item in beta if item.projective_order < b[0].projective_order and item.id > b[0].id ])> 0:
            shift_cost = 0
            shift_case = 1
        else:
            shift_cost = len([d for d in b[0].rdeps if d in stack_ids]) + int(len(s0)>0 and b[0].parent_id in stack_ids[:-1] and b[0].id in b[0].parent_entry.rdeps)
            shift_case = 2


        if len(scores[SWAP]) == 0 :
            swap_cost = 1
        elif s0[0].projective_order > b[0].projective_order:
            swap_cost = 0
            #disable all the others
            left_cost = right_cost = shift_cost = 1
        else:
            swap_cost = 1

        costs = (left_cost, right_cost, shift_cost, swap_cost,1)
        return costs, shift_case


    def oracle_updates(self,best,b,s0,stack_ids,shift_case):
        if best[1] == SHIFT:
            if shift_case == 2:
                if b[0].parent_entry.id in stack_ids[:-1] and b[0].id in b[0].parent_entry.rdeps:
                    b[0].parent_entry.rdeps.remove(b[0].id)
                blocked_deps = [d for d in b[0].rdeps if d in stack_ids]
                for d in blocked_deps:
                    b[0].rdeps.remove(d)

        elif best[1] == LEFT_ARC or best[1] == RIGHT_ARC:
            s0[0].rdeps = []
            if s0[0].id in s0[0].parent_entry.rdeps:
                s0[0].parent_entry.rdeps.remove(s0[0].id)

    def Predict(self, data, datasplit, options, irels):
        reached_max_swap = 0

        pbar = tqdm.tqdm(
            data,
            desc="Parsing",
            unit="sentences",
            mininterval=1.0,
            leave=False,
            disable=False,
        )

        for iSentence, osentence in enumerate(pbar,1):
            config = Configuration(osentence, irels)
            max_swap = 2*len(osentence)
            reached_swap_for_i_sentence = False
            iSwap = 0

            while not config.is_end():
                scores = self.__evaluate(config, False, irels)
                best = max(chain(*(scores if iSwap < max_swap else scores[:3] )), key = itemgetter(2) )
                if iSwap == max_swap and not reached_swap_for_i_sentence:
                    reached_max_swap += 1
                    reached_swap_for_i_sentence = True
                    logging.debug(f"reached max swap in {reached_max_swap:d} out of {iSentence:d} sentences")
                self.apply_transition(best, config)
                if best[1] == SWAP:
                    iSwap += 1

            #keep in memory the information we need, not all the vectors
            oconll_sentence = [entry for entry in osentence if isinstance(entry, ConllEntry)]
            oconll_sentence = oconll_sentence[1:] + [oconll_sentence[0]]
            conll_sentence = config.get_sentence()
            for tok_o, tok in zip(oconll_sentence, conll_sentence):
                tok_o.pred_relation = tok.pred_relation
                tok_o.pred_parent_id = tok.pred_parent_id
            yield osentence

    def cost_computing(self, config, scores, info):
        stack = config.get_stack()
        buf = config.get_buffer()

        stack_ids = [sitem.id for sitem in stack.roots]

        s1 = [stack.roots[-2]] if len(stack) > 1 else []
        s0 = [stack.roots[-1]] if len(stack) > 0 else []
        b = [buf.roots[0]] if len(buf) > 0 else []
        beta = buf.roots[1:] if len(buf) > 1 else []

        costs, shift_case = self.calculate_cost(scores,s0,s1,b,beta,stack_ids)

        bestValid = list(( s for s in chain(*scores) if costs[s[1]] == 0 and ( s[1] == SHIFT or s[1] == SWAP or  s[0] == s0[0].relation ) ))

        bestValid = max(bestValid, key=itemgetter(2))
        bestWrong = max(( s for s in chain(*scores) if costs[s[1]] != 0 or ( s[1] != SHIFT and s[1] != SWAP and s[0] != s0[0].relation ) ), key=itemgetter(2))
        #force swap
        if costs[SWAP]== 0:
            best = bestValid
        else:
        #select a transition to follow
        # + aggresive exploration
        #1: might want to experiment with that parameter
            if bestWrong[1] == SWAP:
                best = bestValid
            else:
                best = bestValid if ( (not self.oracle) or (bestValid[2] - bestWrong[2] > 1.0) or (bestValid[2] > bestWrong[2] and random.random() > 0.1) ) else bestWrong

        #updates for the dynamic oracle
        if self.oracle: # TODO: проверить, что значит True/False
            self.oracle_updates(best,b,s0,stack_ids,shift_case)

        #labeled errors
        if best[1] == LEFT_ARC or best[1] == RIGHT_ARC:
            child = s0[0]
            if (child.pred_parent_id != child.parent_id or child.pred_relation != child.relation):
                info["lerrors"] += 1
                #attachment error
                if child.pred_parent_id != child.parent_id:
                    info["eerrors"] += 1

        if bestValid[2] < bestWrong[2] + 1.0:
            loss = bestWrong[3] - bestValid[3]
            info["mloss"] += 1.0 + bestWrong[2] - bestValid[2]
            info["eloss"] += 1.0 + bestWrong[2] - bestValid[2]
            info["errs"].append(loss)

        #??? when did this happen and why?
        if best[1] == 0 or best[1] == 2:
            info["etotal"] += 1

        return best, info

    def train_sentence(self, sentence, info, irels):
            config = Configuration(sentence, irels)

            ninf = -float('inf')
            while not config.is_end():
                print(config)
                scores = self.__evaluate(config, True, irels)
                scores.append([(None, 4, ninf ,None)]) #to ensure that we have at least one wrong operation

                best, info = self.cost_computing(config, scores, info)

                self.apply_transition(best, config)

            return info

    def error_processing(self, info):
        errs = info["errs"]
        self.labeled_optimizer.zero_grad()
        self.unlabeled_optimizer.zero_grad()
        eerrs = torch.sum(torch.tensor(errs, requires_grad=True))
        eerrs.backward()
        self.labeled_optimizer.step() # TODO Какой из оптимизаторов ???
        self.unlabeled_optimizer.step()
        info["errs"] = []

    def create_info(self):
        info = {}
        info["mloss"], info["eloss"], info["eerrors"], info["lerrors"], info["etotal"]  = 0.0, 0.0, 0, 0, 0
        info["errs"] = []
        info["iSentence"] = -1
        info["start"] = time.time()
        return info

    def train_logging(self, info):
        loss_message = (
            f'Processing sentence number: {info["iSentence"]}'
            f' Loss: {info["eloss"] / info["etotal"]:.3f}'
            f' Errors: {info["eerrors"] / info["etotal"]:.3f}'
            f' Labeled Errors: {info["lerrors"] / info["etotal"]:.3f}'
            f' Time: {time.time()-info["start"]:.3f}s'
        )
        logging.debug(loss_message)
        info["start"] = time.time() # TODO: зачем этот параметр ?
        info["eerrors"], info["eloss"], info["etotal"], info["lerrors"] = 0, 0.0, 0, 0 # TODO: Почему здесь зануляем?

    def Train(self, trainData, options, irels):
        random.shuffle(trainData) # in certain cases the data will already have been shuffled after being read from file or while creating dev data
        logging.info(f"Length of training data: {len(trainData)}")

        beg = time.time()
        info = self.create_info()

        pbar = tqdm.tqdm(
            trainData, desc="Training", unit="sentences",
            mininterval=1.0, leave=False, disable=False,
        )

        for iSentence, sentence in enumerate(pbar,1):
            print("-----------------------------------------------")
            print("Sentence №", iSentence, sentence[2])
            info["iSentence"] = iSentence
            if iSentence % 100 == 0:
                self.train_logging(info)

            info = self.train_sentence(sentence, info, irels)

            #footnote 8 in Eli's original paper
            if len(info["errs"]) > 50: # or True:
                self.error_processing(info)

        if len(info["errs"]) > 0:
            self.error_processing(info)

        logging.info(f"Loss: {info['mloss']/info['iSentence']}")
        logging.info(f"Total Training Time: {time.time()-beg:.2g}s")


# uuparser/parser.py

In [57]:
import pickle, os, time, sys, copy, itertools, re, random

from shutil import copyfile

def evaluate_uas(sentence_descr):
    #sentence_descr is a list, in which elements 0, 1, 2 are auxiliary
    right_parent_tokens = 0
    for token in sentence_descr[3:]:
        if isinstance(token, ConllEntry): # TODO: изучить случаи, когда не ConllEntry - ошибка считывания?
          if token.pred_parent_id == token.parent_id:
              right_parent_tokens += 1
        #print("pred_parent:", token.pred_parent_id, "real_parent:", token.parent_id)
    uas = right_parent_tokens / (len(sentence_descr) - 3)
    return uas

def evaluate_uas_epoche(sentence_list):
    summ_uas = 0
    for sent in sentence_list:
        summ_uas += evaluate_uas(sent)
    return summ_uas / len(sentence_list)

def run(traindata, valdata, testdata, options):

    irels = get_irels(traindata)
    out_irels_dims = len(irels)
    logging.debug('Initializing the model')
    parser = ArcHybridLSTM(options, out_irels_dims)

    dev_best = [options["epochs"],-1.0] # best epoch, best score

    for epoch in range(options["first_epoch"], options["epochs"] + 1):
        # Training
        logging.info(f'Starting epoch {epoch} (training)')
        parser.Train(traindata,options, irels)
        logging.info(f'Finished epoch {epoch} (training)')

        parser.Save(epoch)

        logging.info(f"Predicting on dev data")
        dev_pred = list(parser.Predict(valdata,"dev",options, irels))
        mean_dev_score = evaluate_uas_epoche(dev_pred)
        logging.info(f"Dev score {mean_dev_score:.2f} at epoch {epoch:d}")
        print(f"Dev score {mean_dev_score:.2f} at epoch {epoch:d}")

        if mean_dev_score > dev_best[1]:
            dev_best = [epoch,mean_dev_score] # update best dev score

    logging.info(f"Loading best model from epoche{dev_best[0]:d}")
    # Loading best_models to parser.labeled_GNN and parser.unlabeled_GNN
    parser.Load(epoch)

    logging.info(f"Predicting on test data")

    test_pred = list(parser.Predict(testdata,"test",options, irels))
    mean_test_score = evaluate_uas_epoche(test_pred)

    logging.info(f"On test obtained UAS score of {mean_test_score:.2f}")
    print(f"On test obtained UAS score of {mean_test_score:.2f}")


    logging.debug('Finished predicting')


In [58]:
options = {}
options["hidden_dims"] = 100 # MLP hidden layer dimensions
options["learning_rate"] = 0.001 # Learning rate for neural network optimizer

options["oracle"] = True # Use the static oracle instead of the dynamic oracle

options["epochs"] = 30 # Number of epochs
options["first_epoch"] = 1

# really important to do this before anything else to make experiments reproducible
set_seeds()

train_dir = 'sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu'
val_dir = 'sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu'
test_dir = 'sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-test.conllu'

train, train_words = read_conll(train_dir)
val, val_words = read_conll(val_dir)
test, test_words = read_conll(test_dir)
all_words = train_words + val_words + test_words
all_words = set(all_words)

In [59]:
from transformers import AutoTokenizer, BertModel
def get_embed(tokenizer, model, word):
    inputs = tokenizer(word, return_tensors="pt")
    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state[0][0]
    return last_hidden_states.detach().cpu()

embeds = {}
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

logging.debug('Creating embeddings')
ts = time.time()
for word in all_words:
    embeds[word] = get_embed(tokenizer, model, word)
logging.debug(f'{len(embeds)} embeddings were created')
te = time.time()
logging.info(f'Time of embedding creation: {te-ts:.2g}s')

In [60]:
len(train)

5

In [61]:
print(Path('/content/app.log').read_text())

root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
root - DEBUG - Using default Python seed
root - INFO - Reading sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu
root - DEBUG - 5 sentences read
root - INFO - Time: 0.00065s
root - INFO - Reading sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu
root - DEBUG - 25 sentences read
root - INFO - Time: 0.011s
root - INFO - Reading sample_data/UD_Russian-SynTagRus/ru_syntagrus-ud-test.conllu
root - DEBUG - 27 sentences read
root - INFO - Time: 0.0065s
root - DEBUG - Creating embeddings
root - DEBUG - 516 embeddings were created
root - INFO - Time of embedding creation: 59s



In [62]:
run(train, val, test, options)

Training:   0%|          | 0/5 [00:00<?, ?sentences/s]

-----------------------------------------------
Sentence № 1 # text = Молодая женщина в кокетливой шляпке прошла в кабинет.
stack:
buffer:Молодая 1 женщина 2 в 3 кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1
buffer:женщина 2 в 3 кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1 женщина 2
buffer:в 3 кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1
buffer:в 3 кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1 в 3
buffer:кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1
buffer:кокетливой 4 шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1 кокетливой 4
buffer:шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:Молодая 1
buffer:шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:
buffer:шляпке 5 прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:шляпке 5
buffer:прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:
buffer:прошла 6 в 7 кабинет 8 . 9 *root* 0

stack:прошла

KeyboardInterrupt: ignored

TODO:
В sentence последний элемент -

{'id': 0,
 'form': '*root*',
 'char_rep': '*root*',
 'norm': '*root*',
 'cpos': 'ROOT-CPOS',
 'pos': 'ROOT-POS',
 'parent_id': -1,
 'relation': 'rroot',
 'lemma': '*root*',
 'feats': '_',
 'deps': '_',
 'misc': '_',
 'pred_parent_id': None,
 'pred_relation': None,
 'treebank_id': None,
 'proxy_tbank': None,
 'pred_pos': None,
 'pred_cpos': None,
 'projective_order': 0,
 'rdeps': [8],
 'children': [],
 'scores': None,
 'parent': None,
 'vecs': None}
